# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 20.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=964cc5870fc8f71cfe9e1604dd1b492cfa95a6068fcc714ef4383c397921a36d
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [7]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

products.registerTempTable("products")
orders.registerTempTable("orders")

In [8]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [10]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [65]:
# YOUR CODE HERE
order_baskets = products.select(['product_id', 'product_name'])\
.join(orders.select(['order_id', 'product_id']), on=['product_id']).groupBy('order_id').agg(collect_set('product_name').alias('products_name'))

order_baskets.show()

+--------+--------------------+
|order_id|       products_name|
+--------+--------------------+
|    1342|[Raw Shrimp, Seed...|
|    1591|[Cracked Wheat, S...|
|    4519|[Beet Apple Carro...|
|    4935|             [Vodka]|
|    6357|[Globe Eggplant, ...|
|   10362|[Organic Baby Spi...|
|   19204|[Reduced Fat Crac...|
|   29601|[Organic Red Onio...|
|   31035|[Organic Cripps P...|
|   40011|[Organic Baby Spi...|
|   46266|[Uncured Beef Hot...|
|   51607|[Donut House Choc...|
|   58797|[Concentrated But...|
|   61793|[Raspberries, Gre...|
|   67089|[Original Tofurky...|
|   70863|[Extra Hold Non-A...|
|   88674|[Organic Coconut ...|
|   91937|[No. 485 Gin, Mon...|
|   92317|[Red Vine Tomato,...|
|   99621|[Organic Baby Aru...|
+--------+--------------------+
only showing top 20 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [58]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="products_name", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(order_baskets)

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [59]:
# YOUR CODE HERE
freqItemsets = model.freqItemsets
associationRules = model.associationRules

print(f'Frequent Itemsets: {freqItemsets.count()}')
print(f'Association Rules: {associationRules.count()}')

Frequent Itemsets: 120
Association Rules: 0


Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [66]:
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="products_name", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(order_baskets)

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [67]:
# YOUR CODE HERE
freqItemsets = model.freqItemsets
associationRules = model.associationRules

print(f'Frequent Itemsets: {freqItemsets.count()}')
print(f'Association Rules: {associationRules.count()}')

Frequent Itemsets: 4444
Association Rules: 11


In [68]:
associationRules.show()

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+--------------------+
|[Organic Kiwi, Or...|[Bag of Organic B...|0.5459770114942529| 4.627719489738336|0.001448071397541327|
|[Organic Raspberr...|[Bag of Organic B...|0.5984251968503937| 5.072272070642333|0.001737685677049...|
|[Organic Broccoli...|[Bag of Organic B...|0.5048231511254019| 4.278897986822536|0.001196564260073623|
|[Organic Unsweete...|[Bag of Organic B...|0.5141065830721003| 4.357584667849303|0.001249914258930...|
|[Yellow Onions, S...|            [Banana]|0.5357142857142857|3.7536332219526702|0.001143214261216...|
|[Organic Cucumber...|[Bag of Organic B...|          0.546875| 4.635330870478036|0.001066999977135...|
|[Organic Navel Or...|[Bag of Organic B...|0.5283018867924528| 4.47790453

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!